In [55]:
import sys
sys.path.append('..')
%matplotlib inline
%load_ext autoreload
%autoreload 2
import os
import logging
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt

from ase import Atoms
from ase.io import read

from m_ff.configurations import carve_confs
from m_ff.models import TwoBodySingleSpeciesModel, ThreeBodySingleSpeciesModel, CombinedSingleSpeciesModel
from m_ff.calculators import TwoBodySingleSpecies, ThreeBodySingleSpecies, CombinedSingleSpecies

logging.basicConfig(level=logging.INFO)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Parameters

In [56]:
# Parameters
directory = Path('data/BIP_300/')
r_cut = 10.0
sigma_2b = 0.3
sigma_3b = 0.6
noise = 0.0001

# GP Parameters
ntr = 5
ntest = 5
combine_2b_3b = True

# mapping
grid_start = 1.5
elementslist = [26]
num_2b = 10
num_3b = 10

## Configurations

In [57]:
# # ----------------------------------------
# # Construct a configuration database
# # ----------------------------------------


n_data = 500

filename = directory / 'movie.xyz'

traj = read(filename, index=slice(None), format='extxyz')

elements, confs, forces, energies = carve_confs(
    traj, r_cut, n_data,
    forces_label='force', energy_label='energy')

if not os.path.exists(directory):
    os.makedirs(directory)

np.save('{}/confs_cut={:.2f}.npy'.format(directory, r_cut), confs)
np.save('{}/forces_cut={:.2f}.npy'.format(directory, r_cut), forces)
np.save('{}/energies_cut={:.2f}.npy'.format(directory, r_cut), energies)

lens = [len(conf) for conf in confs]

logging.info('\n'.join((
    'Number of atoms in a configuration:',
    '   maximum: {}'.format(np.max(lens)),
    '   minimum: {}'.format(np.min(lens)),
    '   average: {:.4}'.format(np.mean(lens))
)))

INFO:root:Reading traj step 0
INFO:root:Energy in the xyz file is not present, or is not called energy
INFO:root:Reading traj step 1
INFO:root:Reading traj step 2
INFO:root:Reading traj step 3
INFO:root:Reading traj step 4
INFO:root:Reading traj step 5
INFO:root:Reading traj step 6
INFO:root:Energy in the xyz file is not present, or is not called energy
INFO:root:Reading traj step 7
INFO:root:Reading traj step 8
INFO:root:Reading traj step 9
INFO:root:Reading traj step 10
INFO:root:Reading traj step 11
INFO:root:Reading traj step 12
INFO:root:Energy in the xyz file is not present, or is not called energy
INFO:root:Reading traj step 13
INFO:root:Reading traj step 14
INFO:root:Reading traj step 15
INFO:root:Reading traj step 16
INFO:root:Reading traj step 17
INFO:root:Reading traj step 18
INFO:root:Reading traj step 19
INFO:root:Energy in the xyz file is not present, or is not called energy
INFO:root:Reading traj step 20
INFO:root:Reading traj step 21
INFO:root:Reading traj step 22
INFO:

INFO:root:Energy in the xyz file is not present, or is not called energy
INFO:root:Reading traj step 153
INFO:root:Reading traj step 154
INFO:root:Reading traj step 155
INFO:root:Reading traj step 156
INFO:root:Reading traj step 157
INFO:root:Reading traj step 158
INFO:root:Reading traj step 159
INFO:root:Energy in the xyz file is not present, or is not called energy
INFO:root:Reading traj step 160
INFO:root:Reading traj step 161
INFO:root:Reading traj step 162
INFO:root:Reading traj step 163
INFO:root:Reading traj step 164
INFO:root:Reading traj step 165
INFO:root:Energy in the xyz file is not present, or is not called energy
INFO:root:Reading traj step 166
INFO:root:Reading traj step 167
INFO:root:Reading traj step 168
INFO:root:Reading traj step 169
INFO:root:Reading traj step 170
INFO:root:Reading traj step 171
INFO:root:Energy in the xyz file is not present, or is not called energy
INFO:root:Reading traj step 172
INFO:root:Reading traj step 173
INFO:root:Reading traj step 174
INFO

INFO:root:Reading traj step 303
INFO:root:Reading traj step 304
INFO:root:Reading traj step 305
INFO:root:Energy in the xyz file is not present, or is not called energy
INFO:root:Reading traj step 306
INFO:root:Reading traj step 307
INFO:root:Reading traj step 308
INFO:root:Reading traj step 309
INFO:root:Reading traj step 310
INFO:root:Reading traj step 311
INFO:root:Energy in the xyz file is not present, or is not called energy
INFO:root:Reading traj step 312
INFO:root:Reading traj step 313
INFO:root:Reading traj step 314
INFO:root:Reading traj step 315
INFO:root:Reading traj step 316
INFO:root:Reading traj step 317
INFO:root:Reading traj step 318
INFO:root:Energy in the xyz file is not present, or is not called energy
INFO:root:Reading traj step 319
INFO:root:Reading traj step 320
INFO:root:Reading traj step 321
INFO:root:Reading traj step 322
INFO:root:Reading traj step 323
INFO:root:Reading traj step 324
INFO:root:Energy in the xyz file is not present, or is not called energy
INFO

INFO:root:Reading traj step 453
INFO:root:Reading traj step 454
INFO:root:Reading traj step 455
INFO:root:Reading traj step 456
INFO:root:Reading traj step 457
INFO:root:Reading traj step 458
INFO:root:Energy in the xyz file is not present, or is not called energy
INFO:root:Reading traj step 459
INFO:root:Reading traj step 460
INFO:root:Reading traj step 461
INFO:root:Reading traj step 462
INFO:root:Reading traj step 463
INFO:root:Reading traj step 464
INFO:root:Energy in the xyz file is not present, or is not called energy
INFO:root:Reading traj step 465
INFO:root:Reading traj step 466
INFO:root:Reading traj step 467
INFO:root:Reading traj step 468
INFO:root:Reading traj step 469
INFO:root:Reading traj step 470
INFO:root:Energy in the xyz file is not present, or is not called energy
INFO:root:Reading traj step 471
INFO:root:Reading traj step 472
INFO:root:Reading traj step 473
INFO:root:Reading traj step 474
INFO:root:Reading traj step 475
INFO:root:Reading traj step 476
INFO:root:Rea

INFO:root:Energy in the xyz file is not present, or is not called energy
INFO:root:Reading traj step 605
INFO:root:Reading traj step 606
INFO:root:Reading traj step 607
INFO:root:Reading traj step 608
INFO:root:Reading traj step 609
INFO:root:Reading traj step 610
INFO:root:Energy in the xyz file is not present, or is not called energy
INFO:root:Reading traj step 611
INFO:root:Reading traj step 612
INFO:root:Reading traj step 613
INFO:root:Reading traj step 614
INFO:root:Reading traj step 615
INFO:root:Reading traj step 616
INFO:root:Reading traj step 617
INFO:root:Energy in the xyz file is not present, or is not called energy
INFO:root:Reading traj step 618
INFO:root:Reading traj step 619
INFO:root:Reading traj step 620
INFO:root:Reading traj step 621
INFO:root:Reading traj step 622
INFO:root:Reading traj step 623
INFO:root:Energy in the xyz file is not present, or is not called energy
INFO:root:Reading traj step 624
INFO:root:Reading traj step 625
INFO:root:Reading traj step 626
INFO

...

INFO:root:Energy in the xyz file is not present, or is not called energy
INFO:root:Reading traj step 2545
INFO:root:Reading traj step 2546
INFO:root:Reading traj step 2547
INFO:root:Reading traj step 2548
INFO:root:Reading traj step 2549
INFO:root:Reading traj step 2550
INFO:root:Energy in the xyz file is not present, or is not called energy
INFO:root:Reading traj step 2551
INFO:root:Reading traj step 2552
INFO:root:Reading traj step 2553
INFO:root:Reading traj step 2554
INFO:root:Reading traj step 2555
INFO:root:Reading traj step 2556
INFO:root:Reading traj step 2557
INFO:root:Energy in the xyz file is not present, or is not called energy
INFO:root:Reading traj step 2558
INFO:root:Reading traj step 2559
INFO:root:Reading traj step 2560
INFO:root:Reading traj step 2561
INFO:root:Reading traj step 2562
INFO:root:Reading traj step 2563
INFO:root:Energy in the xyz file is not present, or is not called energy
INFO:root:Reading traj step 2564
INFO:root:Reading traj step 2565
INFO:root:Readi

INFO:root:Reading traj step 2693
INFO:root:Reading traj step 2694
INFO:root:Reading traj step 2695
INFO:root:Reading traj step 2696
INFO:root:Reading traj step 2697
INFO:root:Energy in the xyz file is not present, or is not called energy
INFO:root:Reading traj step 2698
INFO:root:Reading traj step 2699
INFO:root:Reading traj step 2700
INFO:root:Reading traj step 2701
INFO:root:Reading traj step 2702
INFO:root:Reading traj step 2703
INFO:root:Energy in the xyz file is not present, or is not called energy
INFO:root:Reading traj step 2704
INFO:root:Reading traj step 2705
INFO:root:Reading traj step 2706
INFO:root:Reading traj step 2707
INFO:root:Reading traj step 2708
INFO:root:Reading traj step 2709
INFO:root:Energy in the xyz file is not present, or is not called energy
INFO:root:Reading traj step 2710
INFO:root:Reading traj step 2711
INFO:root:Reading traj step 2712
INFO:root:Reading traj step 2713
INFO:root:Reading traj step 2714
INFO:root:Reading traj step 2715
INFO:root:Reading traj

INFO:root:Reading traj step 2843
INFO:root:Energy in the xyz file is not present, or is not called energy
INFO:root:Reading traj step 2844
INFO:root:Reading traj step 2845
INFO:root:Reading traj step 2846
INFO:root:Reading traj step 2847
INFO:root:Reading traj step 2848
INFO:root:Reading traj step 2849
INFO:root:Energy in the xyz file is not present, or is not called energy
INFO:root:Reading traj step 2850
INFO:root:Reading traj step 2851
INFO:root:Reading traj step 2852
INFO:root:Reading traj step 2853
INFO:root:Reading traj step 2854
INFO:root:Reading traj step 2855
INFO:root:Reading traj step 2856
INFO:root:Energy in the xyz file is not present, or is not called energy
INFO:root:Reading traj step 2857
INFO:root:Reading traj step 2858
INFO:root:Reading traj step 2859
INFO:root:Reading traj step 2860
INFO:root:Reading traj step 2861
INFO:root:Reading traj step 2862
INFO:root:Energy in the xyz file is not present, or is not called energy
INFO:root:Reading traj step 2863
INFO:root:Readi

INFO:root:Reading traj step 2990
INFO:root:Reading traj step 2991
INFO:root:Reading traj step 2992
INFO:root:Reading traj step 2993
INFO:root:Reading traj step 2994
INFO:root:Reading traj step 2995
INFO:root:Reading traj step 2996
INFO:root:Energy in the xyz file is not present, or is not called energy
INFO:root:Reading traj step 2997
INFO:root:Reading traj step 2998
INFO:root:Reading traj step 2999
INFO:root:Reading traj step 3000
INFO:root:Reading traj step 3001
INFO:root:Reading traj step 3002
INFO:root:Energy in the xyz file is not present, or is not called energy
INFO:root:Reading traj step 3003
INFO:root:Reading traj step 3004
INFO:root:Reading traj step 3005
INFO:root:Reading traj step 3006
INFO:root:Reading traj step 3007
INFO:root:Reading traj step 3008
INFO:root:Energy in the xyz file is not present, or is not called energy
INFO:root:Reading traj step 3009
INFO:root:Reading traj step 3010
INFO:root:Reading traj step 3011
INFO:root:Reading traj step 3012
INFO:root:Reading traj

INFO:root:Reading traj step 3140
INFO:root:Reading traj step 3141
INFO:root:Reading traj step 3142
INFO:root:Energy in the xyz file is not present, or is not called energy
INFO:root:Reading traj step 3143
INFO:root:Reading traj step 3144
INFO:root:Reading traj step 3145
INFO:root:Reading traj step 3146
INFO:root:Reading traj step 3147
INFO:root:Reading traj step 3148
INFO:root:Energy in the xyz file is not present, or is not called energy
INFO:root:Reading traj step 3149
INFO:root:Reading traj step 3150
INFO:root:Reading traj step 3151
INFO:root:Reading traj step 3152
INFO:root:Reading traj step 3153
INFO:root:Reading traj step 3154
INFO:root:Reading traj step 3155
INFO:root:Energy in the xyz file is not present, or is not called energy
INFO:root:Reading traj step 3156
INFO:root:Reading traj step 3157
INFO:root:Reading traj step 3158
INFO:root:Reading traj step 3159
INFO:root:Reading traj step 3160
INFO:root:Reading traj step 3161
INFO:root:Energy in the xyz file is not present, or is 

In [58]:
# Get configurations and forces from file
confs = np.load(directory / 'confs_cut={:.2f}.npy'.format(r_cut))
forces = np.load(directory / 'forces_cut={:.2f}.npy'.format(r_cut))
numconfs = len(forces)
ind = np.arange(numconfs)
ind_tot = np.random.choice(ind, size=ntr + ntest, replace=False)

# Separate into random testing and training dataset
# tr_confs, tr_forces = confs[ind[:ntr]], forces[ind[:ntr]]
# tst_confs, tst_forces = confs[ind[ntr:]], forces[ind[ntr:]]

# Use fixed training and testing dataset
tr_confs, tr_forces = confs[:ntr], forces[:ntr]
tst_confs, tst_forces = confs[-ntest - 1:-1], forces[-ntest - 1:-1]

## GP - 2+3 body

In [59]:
model_c = CombinedSingleSpeciesModel(elementslist[0], r_cut, sigma_2b, sigma_3b, r_cut / 10.0, r_cut / 10.0, noise)
model_c.fit(tr_confs, tr_forces)

INFO:m_ff.kernels.twobody:Started compilation of theano two body single species kernels
INFO:m_ff.kernels.twobody:Ended compilation of theano two body single species kernels
INFO:m_ff.kernels.threebodykernel:Started compilation of theano three body kernels
INFO:m_ff.kernels.threebodykernel:Ended compilation of theano three body kernels


In [60]:
# Test the GP performance
print('Testing GP')

gp_forces = np.zeros((ntest, 3))
gp_error = np.zeros((ntest, 3))

for i in np.arange(ntest):
    gp_forces[i, :] = model_c.predict(np.reshape(tst_confs[i], (1, len(tst_confs[i]), 5)))
    gp_error[i, :] = gp_forces[i, :] - tst_forces[i, :]

MAEF = np.mean(np.sqrt(np.sum(np.square(gp_error), axis=1)))
SMAEF = np.std(np.sqrt(np.sum(np.square(gp_error), axis=1)))

MF = np.mean(np.linalg.norm(tst_forces, axis=1))

print('MAEF on forces: {:.4f} +- {:.4f}'.format(MAEF, SMAEF))
print('Relative MAEF on forces: {:.4f} +- {:.4f}'.format(MAEF / MF, SMAEF / MF))

Testing GP
MAEF on forces: 0.4274 +- 0.1254
Relative MAEF on forces: 0.7491 +- 0.2198


## Mapping

In [24]:
grid_c_2b, grid_c_3b = model_c.build_grid(grid_start, num_2b, num_3b)

In [26]:
def rmse(x1, x2):
    rms = np.sqrt(np.sum(np.square(x1 - x2), axis=1))
    return np.mean(rms), np.std(rms)

## Test mapping on traj

In [30]:
calc = CombinedSingleSpecies(r_cut, grid_c_2b, grid_c_3b)

filename = directory / 'movie.xyz'
traj = read(str(filename), index=slice(0, 1000))
for atoms in traj:
    atoms.set_cell(100.0*np.identity(3))
    atoms.set_calculator(calc)

    rms_mean, rms_std = rmse(atoms.arrays['force'], atoms.get_forces())
    print('MAEF on forces: {:.4f} +- {:.4f}'.format(rms_mean, rms_std))


INFO:m_ff.calculators:numbers is in system_changes
INFO:m_ff.calculators:initialize


MAEF on forces: 0.0384 +- 0.0224
MAEF on forces: 0.0958 +- 0.0271
MAEF on forces: 0.0951 +- 0.0348
MAEF on forces: 0.0956 +- 0.0416
MAEF on forces: 0.0997 +- 0.0435
MAEF on forces: 0.1054 +- 0.0432
MAEF on forces: 0.1112 +- 0.0419
MAEF on forces: 0.1178 +- 0.0422
MAEF on forces: 0.1252 +- 0.0456
MAEF on forces: 0.1334 +- 0.0507
MAEF on forces: 0.1404 +- 0.0537
MAEF on forces: 0.1455 +- 0.0575
MAEF on forces: 0.1467 +- 0.0585
MAEF on forces: 0.1432 +- 0.0572
MAEF on forces: 0.1366 +- 0.0537
MAEF on forces: 0.1299 +- 0.0486
MAEF on forces: 0.1221 +- 0.0436
MAEF on forces: 0.1147 +- 0.0424
MAEF on forces: 0.1089 +- 0.0447
MAEF on forces: 0.1087 +- 0.0444
MAEF on forces: 0.1124 +- 0.0486
MAEF on forces: 0.1195 +- 0.0549
MAEF on forces: 0.1299 +- 0.0598
MAEF on forces: 0.1423 +- 0.0631
MAEF on forces: 0.1560 +- 0.0656
MAEF on forces: 0.1680 +- 0.0675
MAEF on forces: 0.1783 +- 0.0720
MAEF on forces: 0.1868 +- 0.0783
MAEF on forces: 0.1931 +- 0.0851
MAEF on forces: 0.1965 +- 0.0907
MAEF on fo

MAEF on forces: 0.1413 +- 0.0739
MAEF on forces: 0.1460 +- 0.0740
MAEF on forces: 0.1516 +- 0.0746
MAEF on forces: 0.1576 +- 0.0753
MAEF on forces: 0.1637 +- 0.0748
MAEF on forces: 0.1689 +- 0.0737
MAEF on forces: 0.1728 +- 0.0727
MAEF on forces: 0.1745 +- 0.0731
MAEF on forces: 0.1738 +- 0.0749
MAEF on forces: 0.1707 +- 0.0784
MAEF on forces: 0.1657 +- 0.0831
MAEF on forces: 0.1602 +- 0.0876
MAEF on forces: 0.1556 +- 0.0902
MAEF on forces: 0.1520 +- 0.0897
MAEF on forces: 0.1490 +- 0.0875
MAEF on forces: 0.1465 +- 0.0840
MAEF on forces: 0.1457 +- 0.0801
MAEF on forces: 0.1470 +- 0.0779
MAEF on forces: 0.1505 +- 0.0783
MAEF on forces: 0.1553 +- 0.0801
MAEF on forces: 0.1609 +- 0.0823
MAEF on forces: 0.1666 +- 0.0855
MAEF on forces: 0.1710 +- 0.0890
MAEF on forces: 0.1741 +- 0.0925
MAEF on forces: 0.1766 +- 0.0948
MAEF on forces: 0.1780 +- 0.0959
MAEF on forces: 0.1798 +- 0.0942
MAEF on forces: 0.1814 +- 0.0911
MAEF on forces: 0.1824 +- 0.0874
MAEF on forces: 0.1825 +- 0.0840
MAEF on fo

MAEF on forces: 0.1860 +- 0.0680
MAEF on forces: 0.1781 +- 0.0602
MAEF on forces: 0.1698 +- 0.0561
MAEF on forces: 0.1620 +- 0.0576
MAEF on forces: 0.1563 +- 0.0619
MAEF on forces: 0.1545 +- 0.0653
MAEF on forces: 0.1565 +- 0.0686
MAEF on forces: 0.1607 +- 0.0720
MAEF on forces: 0.1646 +- 0.0771
MAEF on forces: 0.1672 +- 0.0835
MAEF on forces: 0.1685 +- 0.0896
MAEF on forces: 0.1692 +- 0.0939
MAEF on forces: 0.1695 +- 0.0957
MAEF on forces: 0.1686 +- 0.0963
MAEF on forces: 0.1669 +- 0.0964
MAEF on forces: 0.1643 +- 0.0959
MAEF on forces: 0.1608 +- 0.0939
MAEF on forces: 0.1579 +- 0.0910
MAEF on forces: 0.1555 +- 0.0880
MAEF on forces: 0.1539 +- 0.0863
MAEF on forces: 0.1519 +- 0.0861
MAEF on forces: 0.1499 +- 0.0855
MAEF on forces: 0.1479 +- 0.0855
MAEF on forces: 0.1464 +- 0.0849
MAEF on forces: 0.1444 +- 0.0833
MAEF on forces: 0.1423 +- 0.0819
MAEF on forces: 0.1396 +- 0.0808
MAEF on forces: 0.1365 +- 0.0803
MAEF on forces: 0.1335 +- 0.0798
MAEF on forces: 0.1309 +- 0.0796
MAEF on fo

MAEF on forces: 0.2139 +- 0.0774
MAEF on forces: 0.2178 +- 0.0766
MAEF on forces: 0.2176 +- 0.0759
MAEF on forces: 0.2160 +- 0.0753
MAEF on forces: 0.2099 +- 0.0754
MAEF on forces: 0.2019 +- 0.0768
MAEF on forces: 0.1932 +- 0.0793
MAEF on forces: 0.1852 +- 0.0827
MAEF on forces: 0.1791 +- 0.0860
MAEF on forces: 0.1754 +- 0.0896
MAEF on forces: 0.1731 +- 0.0946
MAEF on forces: 0.1720 +- 0.1002
MAEF on forces: 0.1737 +- 0.1036
MAEF on forces: 0.1775 +- 0.1056
MAEF on forces: 0.1823 +- 0.1064
MAEF on forces: 0.1874 +- 0.1058
MAEF on forces: 0.1919 +- 0.1051
MAEF on forces: 0.1957 +- 0.1054
MAEF on forces: 0.1981 +- 0.1074
MAEF on forces: 0.2001 +- 0.1100
MAEF on forces: 0.2026 +- 0.1114
MAEF on forces: 0.2056 +- 0.1119
MAEF on forces: 0.2078 +- 0.1138
MAEF on forces: 0.2101 +- 0.1161
MAEF on forces: 0.2132 +- 0.1175
MAEF on forces: 0.2158 +- 0.1171
MAEF on forces: 0.2171 +- 0.1161
MAEF on forces: 0.2169 +- 0.1143
MAEF on forces: 0.2148 +- 0.1110
MAEF on forces: 0.2103 +- 0.1067
MAEF on fo

## Run simple dynamics

In [39]:
# Global Variables Definition
gamma = 0.001
temp = 300 #K
dt = 3.0  # fs
steps = 1000
t = steps * dt * 10 ** (-3)  # time in ps

In [40]:
atoms = traj[0]
atoms.set_pbc(False)
atoms.set_cell([(21, 0, 0), (0, 21, 0), (0, 0, 21)])
atoms.set_calculator(calc)

In [41]:
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.md.velocitydistribution import Stationary
from ase.md.velocitydistribution import ZeroRotation
from ase.md.verlet import VelocityVerlet
from ase.md.langevin import Langevin
from ase import units


Stationary(atoms)
ZeroRotation(atoms)
dyn = VelocityVerlet(atoms, dt * units.fs)
MaxwellBoltzmannDistribution(atoms, temp * units.kB)

In [42]:

dyn.run(steps)

In [47]:
from ase.calculators.eam import EAM

from ase.build import bulk

# test to generate an EAM potential file using a simplified
# approximation to the Mishin potential Al99.eam.alloy data

from scipy.interpolate import InterpolatedUnivariateSpline as spline

cutoff = 10.0

n = 21
rs = np.arange(0, n) * (cutoff / n)
rhos = np.arange(0, 2, 2. / n)

# generated from
# mishin = EAM(potential='../potentials/Al99.eam.alloy')
# m_density = mishin.electron_density[0](rs)
# m_embedded = mishin.embedded_energy[0](rhos)
# m_phi = mishin.phi[0,0](rs)

m_density = np.array([2.78589606e-01, 2.02694937e-01, 1.45334053e-01,
                      1.06069912e-01, 8.42517168e-02, 7.65140344e-02,
                      7.76263116e-02, 8.23214224e-02, 8.53322309e-02,
                      8.13915861e-02, 6.59095390e-02, 4.28915711e-02,
                      2.27910928e-02, 1.13713167e-02, 6.05020311e-03,
                      3.65836583e-03, 2.60587564e-03, 2.06750708e-03,
                      1.48749693e-03, 7.40019174e-04, 6.21225205e-05])

m_embedded = np.array([1.04222211e-10, -1.04142633e+00, -1.60359806e+00,
                       -1.89287637e+00, -2.09490167e+00, -2.26456628e+00,
                       -2.40590322e+00, -2.52245359e+00, -2.61385603e+00,
                       -2.67744693e+00, -2.71053295e+00, -2.71110418e+00,
                       -2.69287013e+00, -2.68464527e+00, -2.69204083e+00,
                       -2.68976209e+00, -2.66001244e+00, -2.60122024e+00,
                       -2.51338548e+00, -2.39650817e+00, -2.25058831e+00])

m_phi = np.array([6.27032242e+01, 3.49638589e+01, 1.79007014e+01,
                  8.69001383e+00, 4.51545250e+00, 2.83260884e+00,
                  1.93216616e+00, 1.06795515e+00, 3.37740836e-01,
                  1.61087890e-02, -6.20816372e-02, -6.51314297e-02,
                  -5.35210341e-02, -5.20950200e-02, -5.51709524e-02,
                  -4.89093894e-02, -3.28051688e-02, -1.13738785e-02,
                  2.33833655e-03, 4.19132033e-03, 1.68600692e-04])

m_densityf = spline(rs, m_density)
m_embeddedf = spline(rhos, m_embedded)
m_phif = spline(rs, m_phi)

mishin_approx = EAM(elements=['Ni'], embedded_energy=np.array([m_embeddedf]),
                    electron_density=np.array([m_densityf]),
                    phi=np.array([[m_phif]]), cutoff=cutoff, form='alloy',
                    # the following terms are only required to write out a file
                    Z=[13], nr=n, nrho=n, dr=cutoff / n, drho=2. / n,
                    lattice=['fcc'], mass=[58.692], a=[2.58])

atoms.set_calculator(calc)

In [48]:
dyn.run(steps)

## Pairwise potential test

In [50]:
model_2b = TwoBodySingleSpeciesModel(elementslist[0], r_cut, sigma_2b, r_cut / 10.0, noise)
model_2b.fit(tr_confs, tr_forces)

INFO:m_ff.kernels.twobody:Started compilation of theano two body single species kernels
INFO:m_ff.kernels.twobody:Ended compilation of theano two body single species kernels


In [51]:
# Test the GP performance
print('Testing GP')

gp_forces = np.zeros((ntest, 3))
gp_error = np.zeros((ntest, 3))

for i in np.arange(ntest):
    gp_forces[i, :] = model_2b.predict(np.reshape(tst_confs[i], (1, len(tst_confs[i]), 5)))
    gp_error[i, :] = gp_forces[i, :] - tst_forces[i, :]

MAEF = np.mean(np.sqrt(np.sum(np.square(gp_error), axis=1)))
SMAEF = np.std(np.sqrt(np.sum(np.square(gp_error), axis=1)))

MF = np.mean(np.linalg.norm(tst_forces, axis=1))

print('MAEF on forces: {:.4f} +- {:.4f}'.format(MAEF, SMAEF))
print('Relative MAEF on forces: {:.4f} +- {:.4f}'.format(MAEF / MF, SMAEF / MF))

Testing GP
MAEF on forces: 0.1404 +- 0.0750
Relative MAEF on forces: 0.1943 +- 0.1038


In [52]:
grid_2b = model_2b.build_grid(grid_start, num_2b)


In [53]:
calc = TwoBodySingleSpecies(r_cut, grid_c_2b)
atoms.set_calculator(calc)

In [54]:
dyn.run(steps)

INFO:m_ff.calculators:numbers is in system_changes
INFO:m_ff.calculators:initialize
